In [1]:
from pypdf import PdfReader
import cohere  
import io
import os

In [2]:
co = cohere.Client(os.environ.get("COHERE_API_KEY", None))

In [6]:
# Read from PDF

reader = PdfReader("data/walden.pdf")
# reader = PdfReader("data/memoriasBras.pdf")

page = reader.pages[0]
# print(page.extract_text())

pages_list = [p for p in reader.pages]
first_pages_content = " ".join([p.extract_text().replace("\n", " ").replace("..", "") for p in pages_list[0:4]])[0:3000]

In [14]:
full_text = " ".join([p.extract_text().replace("\n", " ").replace("..", "") for p in pages_list])

In [22]:
sentences = full_text.split(". ")
sentences_list = list()
last_sentence = sentences[0]
for i, s in enumerate(sentences[1:]):
    combined_sentence = last_sentence + ". " + s
    if len(combined_sentence) > 2000:
        sentences_list.append(last_sentence)
        last_sentence = s
    else:
        last_sentence = combined_sentence
        if i == len(sentences) - 2:
            sentences_list.append(last_sentence)
        
sentences_list[-1]

'Every one has heard the story which has gone the rounds of New England, of a strong and beautiful bug which came out of the dry leaf of an old table of apple-tree wood, which had stood in a farmer’s kitchen for sixty years, first in Connecticut, and afterward in Massachusetts,—from an egg deposited in the living tree many years earlier still, as appeared by counting the annual layers beyond it; which was heard gnawing out for several weeks, hatched perchance by the heat of an urn. Who does not feel his faith in a resurrection and immortality strengthened by hearing of this? Who knows what beautiful and winged life, whose egg has been buried for ages under many concentric layers of woodenness in the dead dry life of society, deposited at first in the alburnum of the green and living tree, which has been gradually converted into the semblance of its well-seasoned tomb,—heard perchance gnawing out now for years by the astonished family of man, as they sat round the festive board,—may une

In [94]:
# Read from TXT
import chardet

# Open the text file in read-binary mode
with open('data/james.txt', 'rb') as file:
    
    # Detect the encoding of the file
    file_encoding = chardet.detect(file.read())['encoding']
    
    # Reopen the file with the detected encoding in read mode
    file.seek(0)
    file_contents = file.read().decode(file_encoding)
    
first_pages_content = file_contents.replace("\n", " ").replace("\r", " ").replace("..", "")[0:2000]

In [139]:
prompt = f"""  
Given the text below, extract these three pieces of information: 
1) publication title: 
2) author: 
3) publication year: (should be just a number)

Text:
{first_pages_content}

Information:
"""

prompt

'  \nGiven the text below, extract these three pieces of information: \n1) publication title: \n2) author: \n3) publication year: (should be just a number)\n\nText:\n   Memórias Póstumas de Brás Cubas           Texto-fonte:   Obra Completa , Machado de Assis,   Rio de Janeiro: Editora Nova Aguilar, 1994.      Publicado originalmente em folhetins, a partir de março de 1880, na Revista Brasileira .                        Ao verme   que   primeiro roeu as frias carnes   do meu cadáver   dedico   como saudosa lembrança   estas   Memórias Póstumas             Prólogo da terceira edição      A primeira edição destas Memórias Póstumas  de Brás Cubas  foi feita  aos pedaços na Revista Brasileira, pelos anos de 1880. Postas mais  tarde em livro, corrigi o texto em vá rios lugares. Agora que tive de o  rever para a terceira edição, emendei ainda alguma coisa e suprimi  duas ou três dúzias de linhas. Assim composta, sai novamente à luz  esta obra que alguma benevolência parece ter encontrado no p

In [87]:
response = co.generate(  
    model='xlarge',  
    prompt = prompt,  
    max_tokens=40,  
    temperature=0.2,  
    stop_sequences=["--"])

info = response.generations[0].text

In [88]:
info

'\n1) book title: The Will to Believe\n\n2) author: William James\n\n3) publication year: 1912\n\n4) publication place: New York\n\n5) publisher'

In [68]:
first_pages_content

"The Project Gutenberg EBook of The Will to Believe, by William James    This eBook is for the use of anyone anywhere at no cost and with  almost no restrictions whatsoever.  You may copy it, give it away or  re-use it under the terms of the Project Gutenberg License included  with this eBook or online at www.gutenberg.org      Title: The Will to Believe         and Other Essays in Popular Philosophy    Author: William James    Release Date: May 8, 2009 [EBook #26659]    Language: English    Character set encoding: ISO-8859-1    *** START OF THIS PROJECT GUTENBERG EBOOK THE WILL TO BELIEVE ***          Produced by Al Haines. (This file was produced from images  generously made available by The Internet Archive.)                      THE WILL TO BELIEVE      AND OTHER ESSAYS IN    POPULAR PHILOSOPHY        BY WILLIAM JAMES          NEW IMPRESSION        LONGMANS, GREEN, AND CO.    FOURTH AVENUE & 30TH STREET, NEW YORK    LONDON, BOMBAY, AND CALCUTTA    1912          _Copyright, 1896_   

In [48]:
response = co.detect_language(
  texts=["testando esse negocio em uma lingua nova"],
)
response

# Using OpenAI api

In [118]:
import openai

In [140]:
messages = [{"role": "user", "content": prompt}]

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
  temperature=0,
  max_tokens=100,
  # frequency_penalty=0.0,
  # presence_penalty=0.0,
  # stop=["\n"]
)

In [144]:
response["choices"][0]["message"]["content"].lower()

'1) publication title: memórias póstumas de brás cubas\n2) author: machado de assis\n3) publication year: 1994'

In [147]:
response_string = response["choices"][0]["message"]["content"].lower()
response_string.split("title:")[1].split("author:")[0].split("\n")[0].strip()
# response_string.split("author:")[1].split("\n")[0].strip()
# response_string.split("year:")[1].split("\n")[0].strip()

'memórias póstumas de brás cubas'

# Auto question

In [97]:
first_pages_content = file_contents.replace("\n", " ").replace("\r", " ").replace("..", "")[5000:7000]
first_pages_content

"ypothesis the notion that it is the permanent form of the world is  what I call a radical empiricist.  For him the crudity of experience  remains an eternal element thereof.  There is no possible point of view  from which the world can appear an absolutely single fact.  Real  possibilities, real indeterminations, real beginnings, real ends, real  evil, real crises, catastrophes, and escapes, a real God, and a real  moral life, just as common-sense conceives these things, may remain in  empiricism as conceptions which that philosophy gives up the attempt  either to 'overcome' or to reinterpret in monistic form.    Many of my professionally trained _confrères_ will smile at the  irrationalism of this view, and at the artlessness of my essays in  point of technical form.  But they should be taken as illustrations of  the radically empiricist attitude rather than as argumentations for its  validity.  That admits meanwhile of {x} being argued in as technical a  shape as any one can desire,

In [114]:
prompt = f"""  
Given the text below, come up with just one question about it. 
Make sure to include the authors's name and book title in the question.
Do not answer it, just create one question.

book title: The Will to Believe
author: William James

Text:
{first_pages_content}

Question:
"""

In [115]:
response = co.generate(  
    model='xlarge',  
    prompt = prompt,  
    max_tokens=40,  
    temperature=0.2,  
    stop_sequences=["--"])

info = response.generations[0].text

In [116]:
info

"What is the author's view of the world?\nA. The world is a single fact.\nB. The world is a single fact, but it is not a fact.\nC."

# Qdrant

In [23]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

In [ ]:
qdrant_client = QdrantClient("localhost", port=6333)

qdrant_client = QdrantClient(
    host=os.eviron.get("QDRANT_HOST", None),
    api_key=os.eviron.get("QDRANT_API_KEY", None),
)